In [ ]:
import pandas as pd
from typing import Dict, Set
import seaborn as sns
import matplotlib.pyplot as plt

from tqdm import tqdm

tqdm.pandas()

base on the last draw, we will filter data from 2019 to 2023, and keep only user who actively invested in all the years from 2019 to 2023.

In [ ]:
df = pd.read_parquet("../data/gen/preprocessed_2023-08-28T11-09-39.parquet")

In [ ]:
df = df[df["fundraisingDate"] >= "2019-01-01"]

In [ ]:
each_year = df.groupby(df["fundraisingDate"].dt.year).agg({"lender_id": set})
each_year["lender_count"] = each_year["lender_id"].apply(len)
each_year.reset_index(inplace=True)
each_year.tail()

In [ ]:
union = set()
for s in each_year["lender_id"]:
    union = union.union(s)

len(union)

In [ ]:
intersec = union
for s in each_year["lender_id"]:
    intersec = intersec.intersection(s)

len(intersec)

In [ ]:
len(intersec) / len(union)

In [ ]:
active_lender = intersec

In [ ]:
# how many project that these active lenders have funded?
df[df["lender_id"].isin(active_lender)]["project_id"].nunique()

In [ ]:
# how many total projects?
df["project_id"].nunique()

In [ ]:
# how many loans that these active lenders have funded?
df[df["lender_id"].isin(active_lender)]["loan_id"].nunique()

In [ ]:
# how many loans in total?
df["loan_id"].nunique()

In [ ]:
7368409 / 11128507

In [ ]:
# keep only loan that has active lenders
df = df[df["lender_id"].isin(active_lender)]

In [ ]:
# save it, prepare to lunch biLouvian again
df.to_parquet("../data/gen/preprocessed_2023-08-28T11-09-39_from_2019-01-01_activelender.parquet")